In [1]:
# Note: need to run the following in powershell
#       pip install polygon-api-client

import pandas as pd
from polygon import RESTClient

# Note: the furthest back we can go with the free plan seems to be exactly 2 years from the current date
key = 'TUo8bKPGXbU91jEOuI_Uus5EL73Q9rBC'
client = RESTClient(key)

In [2]:
from datetime import datetime

# Note:  the API lists timestamps rather than dates.
#        These are unix millisecond timestamps, which describe the number of milliseconds 
#        since Jan 1, 1970, midnight UTC/GMT.
#        We use the following function to convert timestamps to date strings

def timestamp_to_date(timestamp):
    """
    Convert a UNIX timestamp in milliseconds to a date string in the format YYYY-MM-DD.
    """
    
    # Convert to seconds and shift by a day to align with data
    epoch_secs = timestamp/1000 + 86400
    
    # Convert the timestamp to a datetime object
    dt_object = datetime.fromtimestamp(epoch_secs)
    
    # Convert to string of form YYYY-MM-DD
    date_string = dt_object.strftime('%Y-%m-%d')
    
    return date_string

In [3]:
def multiple_stock_table(date):
    """
    Returns a dataframe containing all info on all stocks at a single specified date
    Date is a string of the form YYYY-MM-DD
    Dataframe is sorted by trade volume
    """
    
    data = client.get_grouped_daily_aggs(date)

    df = pd.DataFrame({
        'Close' : [stock.close for stock in data],
        'Volume': [stock.volume for stock in data],
        'Open' : [stock.open for stock in data],
        'High' : [stock.high for stock in data],
        'Low' : [stock.low for stock in data], 
        'Vwap': [stock.vwap for stock in data],
        'Transactions': [stock.transactions for stock in data]},
        index = [stock.ticker for stock in data])
    
    df = df.sort_values('Volume', ascending = False)
    
    return df


In [6]:
df = multiple_stock_table('2024-02-20')
df

,Close,Volume,Open,High,Low,Vwap,Transactions
NVDA,69.4520,704774600.0,71.9470,71.9560,67.7340,69.0538,1568243.0
SMCI,78.7570,254098560.0,79.0000,80.2000,69.2500,74.5995,628145.0
TSLA,193.7600,104537762.0,196.1300,198.6000,189.1300,192.5316,1220065.0
SOUN,3.9900,101328674.0,4.0900,4.3500,3.6600,3.9478,253250.0
PLTR,23.4000,93374349.0,23.8400,24.0000,22.7150,23.2671,365494.0
...,...,...,...,...,...,...,...
MNBD,25.8200,0.0,25.8200,25.8200,25.8200,NaN,NaN
RHCB,45.5687,0.0,45.5687,45.5687,45.5687,NaN,NaN
BBBL,49.0523,0.0,49.0523,49.0523,49.0523,NaN,NaN
OCTZ,34.5087,0.0,34.5087,34.5087,34.5087,NaN,NaN


In [21]:
def single_stock_table(ticker, start_date, end_date):
    """
    Returns a dataframe containing all info on a given stock from start_date to end_date
    Dates are strings of the form YYYY-MM-DD
    """
    
    data = client.get_aggs(ticker, 1, 'day', start_date, end_date)
    
    df = pd.DataFrame({
    'Close' : [entry.close for entry in data],
    'Volume': [entry.volume for entry in data],
    'Open' : [entry.open for entry in data],
    'High' : [entry.high for entry in data],
    'Low' : [entry.low for entry in data], 
    'Vwap': [entry.vwap for entry in data],
    'Transactions': [entry.transactions for entry in data],
    'Timestamp' : [entry.timestamp for entry in data]},
    index = [timestamp_to_date(entry.timestamp) for entry in data]
    )
    
    return df



In [22]:
df = single_stock_table('NVDA', '2022-08-01', '2025-02-12')
df

,Close,Volume,Open,High,Low,Vwap,Transactions,Timestamp
2023-02-21,20.655,4.100146e+08,21.000,21.4938,20.618,20.9405,427159,1676955600000
2023-02-22,20.754,5.130883e+08,20.707,21.1040,20.421,21.0021,542700,1677042000000
2023-02-23,23.664,1.117995e+09,23.440,23.8880,23.025,23.5205,1259505,1677128400000
2023-02-24,23.286,5.896727e+08,23.225,23.4740,22.947,23.2086,671466,1677214800000
2023-02-27,23.501,4.529937e+08,23.670,23.8799,23.454,23.6499,508293,1677474000000
...,...,...,...,...,...,...,...,...
2025-02-06,128.680,2.481460e+08,127.420,128.7700,125.210,127.3319,1855949,1738818000000
2025-02-07,129.840,2.266308e+08,129.220,130.3700,125.000,129.0634,1778758,1738904400000
2025-02-10,133.570,2.113588e+08,130.090,135.0000,129.960,133.6341,1658255,1739163600000
2025-02-11,132.800,1.753496e+08,132.580,134.4800,131.020,133.1308,1292554,1739250000000


In [9]:
import requests 

def fetch_macd(stock_ticker, timespan="day", short_window=12, long_window=26, signal_window=9, limit=10):
    """
    Fetch MACD data from Polygon API for a given stock.

    If the MACD is higher than the signal line there is a upward trend and should think of buying 
    If the MACD is lower than the signal line there is a downwards trend and should think of selling
    """
    url = f"https://api.polygon.io/v1/indicators/macd/{stock_ticker}?timespan={timespan}&adjusted=true&short_window={short_window}&long_window={long_window}&signal_window={signal_window}&series_type=close&order=desc&limit={limit}&apiKey={key}"
    
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error fetching MACD data: {response.json()}")

    data = response.json()
    
    if "results" not in data or "values" not in data["results"]:
        raise Exception("No MACD data available")

    macd_values = data["results"]["values"]
    df = pd.DataFrame(macd_values)[["timestamp", "value", "signal"]]
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.rename(columns={"value": "MACD", "signal": "Signal_Line"}, inplace=True)
    
    return df


macd_df = fetch_macd("AAPL")
macd_df

,timestamp,MACD,Signal_Line
0,2025-02-18 05:00:00,0.780017,-1.165477
1,2025-02-14 05:00:00,0.069752,-1.651850
2,2025-02-13 05:00:00,-0.851055,-2.082251
3,2025-02-12 05:00:00,-1.701379,-2.390049
4,2025-02-11 05:00:00,-2.287482,-2.562217
5,2025-02-10 05:00:00,-2.569563,-2.630901
6,2025-02-07 05:00:00,-2.386189,-2.646235
7,2025-02-06 05:00:00,-2.115392,-2.711246
8,2025-02-05 05:00:00,-2.308152,-2.860210
9,2025-02-04 05:00:00,-2.444246,-2.998225


In [10]:
def stock_proc(ticker, start_date, end_date, n):
    """
    Fetch stock data for a given ticker between start_date and end_date,
    and compute the Price Rate of Change (PROC) over the last 'n' days.
    Automatically fixes incorrect date order.
    """
    # Ensure correct date order
    start_date, end_date = min(start_date, end_date), max(start_date, end_date)

    # Fetch stock data from API
    data = client.get_aggs(ticker, 1, 'day', start_date, end_date)
    
    # Convert data into a DataFrame
    df = pd.DataFrame({
        'Close': [entry.close for entry in data],
        'Volume': [entry.volume for entry in data],
        'Open': [entry.open for entry in data],
        'High': [entry.high for entry in data],
        'Low': [entry.low for entry in data], 
        'Vwap': [entry.vwap for entry in data],
        'Transactions': [entry.transactions for entry in data]
    }, index=[timestamp_to_date(entry.timestamp) for entry in data])

    # Ensure data is sorted by date
    df.sort_index(inplace=True)
    
    # Compute PROC
    df["Close_n_days_ago"] = df["Close"].shift(n)  # Closing price 'n' days ago
    df["PROC"] = ((df["Close"] - df["Close_n_days_ago"]) / df["Close_n_days_ago"]) * 100
    
    # Get the latest row with valid PROC
    latest_data = df.iloc[-1][["Close", "Close_n_days_ago", "PROC"]]
    
    # Return the final DataFrame
    result = pd.DataFrame({
        "Ticker": [ticker],
        "Current Close": [latest_data["Close"]],
        f"Close {n} Days Ago": [latest_data["Close_n_days_ago"]],
        f"PROC ({n} days)": [latest_data["PROC"]]
    })
    
    return result


In [11]:
df = stock_proc("NVDA", "2025-02-09", "2025-01-10", 10)
df

,Ticker,Current Close,Close 10 Days Ago,PROC (10 days)
0,NVDA,129.84,142.62,-8.960875


In [13]:
def fetch_rsi(stock_ticker, timespan="day", window=14, limit=10):
    """
    Fetches the Relative Strength Index (RSI) for a given stock ticker.
    RSI > 70 → Overbought (possible price decrease)
    RSI < 30 → Oversold (possible price increase)
    """
    url = f"https://api.polygon.io/v1/indicators/rsi/{stock_ticker}"

    params = {
        "timespan": timespan,
        "adjusted": "true",
        "window": window,
        "series_type": "close",
        "order": "desc",
        "limit": limit,
        "apiKey": key
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.json()}")
        return None

    data = response.json()

    if "results" not in data or "values" not in data["results"]:
        print("No RSI data found.")
        return None

    # Convert response to DataFrame
    rsi_data = pd.DataFrame(data["results"]["values"])
    rsi_data["timestamp"] = pd.to_datetime(rsi_data["timestamp"], unit='ms')  # Convert timestamp to datetime
    rsi_data.rename(columns={"value": "RSI"}, inplace=True)

    return rsi_data

rsi_df = fetch_rsi("AAPL", limit=10)

rsi_df

,timestamp,RSI
0,2025-02-18 05:00:00,60.330653
1,2025-02-14 05:00:00,60.512841
2,2025-02-13 05:00:00,57.712551
3,2025-02-12 05:00:00,53.016230
4,2025-02-11 05:00:00,48.138601
5,2025-02-10 05:00:00,41.549401
6,2025-02-07 05:00:00,41.521637
7,2025-02-06 05:00:00,47.360021
8,2025-02-05 05:00:00,46.421435
9,2025-02-04 05:00:00,46.762095


In [14]:
def range_stock_data(start_date, end_date, stock_ticker):
    """
    Fetches historical stock data (High, Low, Close) over a date range.
    
    """
    data = client.get_aggs(
        ticker=stock_ticker,
        multiplier=1,
        timespan="day",
        from_=start_date,
        to=end_date,
        adjusted=True,
        sort="asc",
        limit=5000
    )

    df = pd.DataFrame({
        'Timestamp': [stock.timestamp for stock in data],
        'Close': [stock.close for stock in data],
        'Volume': [stock.volume for stock in data],
        'Open': [stock.open for stock in data],
        'High': [stock.high for stock in data],
        'Low': [stock.low for stock in data], 
        'Vwap': [stock.vwap for stock in data],
        'Transactions': [stock.transactions for stock in data]},
    )

    df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='ms')  # Convert timestamp to datetime
    df = df.sort_values('Timestamp', ascending=True)  # Sort data by time

    return df


In [15]:
df = range_stock_data("2024-01-01", "2024-02-15", "AAPL")
df

,Timestamp,Close,Volume,Open,High,Low,Vwap,Transactions
0,2024-01-02 05:00:00,185.64,81964874.0,187.150,188.4400,183.8850,185.9465,1008871
1,2024-01-03 05:00:00,184.25,58414460.0,184.220,185.8800,183.4300,184.3226,656853
2,2024-01-04 05:00:00,181.91,71878670.0,182.150,183.0872,180.8800,182.0183,712692
3,2024-01-05 05:00:00,181.18,62371161.0,181.990,182.7600,180.1700,181.4740,682334
4,2024-01-08 05:00:00,185.56,59144470.0,182.085,185.6000,181.5000,184.3702,669173
5,2024-01-09 05:00:00,185.14,42841809.0,183.920,185.1500,182.7300,184.3706,538180
6,2024-01-10 05:00:00,186.19,46192908.0,184.350,186.4000,183.9200,185.2509,554777
7,2024-01-11 05:00:00,185.59,49128408.0,186.540,187.0500,183.6200,185.0604,584008
8,2024-01-12 05:00:00,185.92,40477782.0,186.060,186.7400,185.1900,185.8199,477050
9,2024-01-16 05:00:00,183.63,65076641.0,182.160,184.2600,180.9340,182.8866,767281


In [24]:
def calculate_williams_r(stock_data, window=14):
    """
    Calculates Williams %R for the given stock data.
    Above -20 → Sell signal
    Below -80 → Buy signal
    """
    stock_data = stock_data.sort_values("Timestamp", ascending=True)

    # Calculate Williams %R using a rolling window
    stock_data["Williams %R"] = stock_data.apply(
        lambda row: (
            (max(stock_data.loc[row.name - window + 1: row.name, "High"]) - row["Close"]) /
            (max(stock_data.loc[row.name - window + 1: row.name, "High"]) - 
             min(stock_data.loc[row.name - window + 1: row.name, "Low"]))
        ) * -100 if row.name >= window - 1 else None, axis=1
    )

    return stock_data

In [25]:
stock_data = single_stock_table("AAPL", "2024-01-01", "2024-02-15")
df = calculate_williams_r(stock_data)
df.dropna()

TypeError: '>=' not supported between instances of 'str' and 'int'